In [1]:
from bayes_opt import BayesianOptimization
import xgboost

from helpers.utils import *
from helpers.xgboost_feval import *

In [2]:
DIR = "/Users/brianbroeking/projects/numerai/data"
download_current_data(DIR)
train_static, val_static, tournament_static = load_data(DIR, reduce_memory=True)
features_list = generate_features_list(train_static)
train, val, tournament = get_group_stats(train_static), get_group_stats(val_static), get_group_stats(tournament_static)

You already have the newest data! Current round is: 261
Loading the data


/opt/homebrew/Caskroom/miniforge/base/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/opt/homebrew/Caskroom/miniforge/base/lib/python3.8/site-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [4]:
# ft_corr_list = ['feature_intelligence1', 'feature_constitution2', 'feature_charisma71', 'feature_wisdom45']
# train, val, tournament = generate_polynomial_features(ft_corr_list, (train, val, tournament))

In [3]:
X_train, y_train = clean_for_xgboost(train)
X_val, y_val = clean_for_xgboost(val)
X_tournament, y_tournament = clean_for_xgboost(tournament)

In [4]:
X_train.columns = X_train.columns.astype(str)

In [5]:
dtrain = xgb.DMatrix(X_train, y_train, feature_names=X_train.columns.values)

In [6]:
def correlation_score(preds, dtrain):
    labels = dtrain.get_label()
    return 'corr', np.corrcoef(labels, preds)[0,1]

In [7]:
def hyp_xgb(max_depth, min_child_weight, gamma, l2, l1, learning_rate):
    params = {
      'objective': 'reg:squarederror',
      'colsample_bytree': 0.1,
      'n_estimators': 25,
      'verbosity': 0
    }
    params['max_depth'] = int(round(max_depth))
    params['min_child_weight'] = int(min_child_weight)
    params['gamma'] = max(gamma, 0)
    params['lambda'] = max(l2, 0)
    params['alpha'] = max(l1, 0)
    params['eta'] = max(learning_rate, 0)
#     parmas['n_estimators'] = max(n_estimators, 0)
    
    scores = xgboost.cv(params,
                    dtrain,
                    num_boost_round=10,
                    verbose_eval=False,
                    early_stopping_rounds=5,
                    feval=correlation_score,
                    maximize=True,
                    nfold=5)
    return  scores['test-corr-mean'].iloc[-1]

In [8]:
pds = {
  'max_depth': (4, 10),
  'min_child_weight':(14, 20),
  'gamma': (0, 1),
  'l2': (1, 10),
  'l1': (0, 10),
  'learning_rate': (0.001, 0.3),
}

In [9]:
params_dist_grid = {
    'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'gamma': [0, 0.5, 1],
    'learning_rate': uniform(), # gaussian distribution
    'subsample': uniform(), # gaussian distribution
    'colsample_bytree': uniform() # gaussian distribution
}

NameError: name 'uniform' is not defined

In [47]:
# Surrogate model
optimizer = BayesianOptimization(hyp_xgb, pds, random_state=101)
                                  
# Optimize
optimizer.maximize(init_points=5, n_iter=15)

|   iter    |  target   |   gamma   |    l1     |    l2     | learni... | max_depth | min_ch... |
-------------------------------------------------------------------------------------------------
|  1        |  0.03719  |  0.5164   |  5.707    |  1.256    |  0.05228  |  8.112    |  19.0     |
|  2        |  0.03899  |  0.307    |  8.936    |  7.494    |  0.05779  |  7.325    |  16.11    |
|  3        |  0.0383   |  0.1819   |  7.856    |  9.689    |  0.07047  |  4.501    |  17.62    |
|  4        |  0.03555  |  0.729    |  2.762    |  7.168    |  0.1558   |  4.291    |  14.83    |
|  5        |  0.03596  |  0.187    |  9.943    |  5.686    |  0.1741   |  8.409    |  17.25    |
|  6        |  0.03557  |  0.6588   |  2.45     |  5.531    |  0.1521   |  8.685    |  19.65    |
|  7        |  0.03529  |  0.954    |  1.381    |  2.417    |  0.2017   |  5.418    |  15.28    |
|  8        |  0.03598  |  0.5711   |  4.591    |  1.071    |  0.1913   |  9.242    |  14.12    |
|  9        |  0.037

In [ ]:
# Surrogate model
optimizer = BayesianOptimization(hyp_xgb, pds)
                                  
# Optimize
optimizer.maximize(init_points=5, n_iter=15)

In [30]:
optimizer.max

{'target': 0.0342492,
 'params': {'colsample_bytree': 0.33649076554038165,
  'gamma': 0.782158881148831,
  'max_depth': 5.847339640297226,
  'min_child_weight': 19.430576015452555,
  'subsample': 1.0}}

In [16]:
params = {
    'eta': 0.05,
    'objective': 'reg:squarederror',
    'verbosity': 0,
    'max_depth': 5,
    'min_child_weight': 14,
    'gamma': 0,
    'colsample_bytree': 0.1,
    "subsample": 0.5
}
scores2 = xgboost.cv(params,
                dtrain,
                num_boost_round=1000,
                verbose_eval=True,
                early_stopping_rounds=5,
                feval=correlation_score,
                maximize=True,
                nfold=5)

[0]	train-logloss:0.69319+0.00001	train-corr:-0.03479+0.00153	test-logloss:0.69319+0.00000	test-corr:-0.01719+0.00412
[1]	train-logloss:0.69316+0.00001	train-corr:-0.04199+0.00136	test-logloss:0.69319+0.00000	test-corr:-0.02024+0.00427
[2]	train-logloss:0.69314+0.00000	train-corr:-0.04790+0.00111	test-logloss:0.69318+0.00000	test-corr:-0.02343+0.00262
[3]	train-logloss:0.69313+0.00000	train-corr:-0.05212+0.00129	test-logloss:0.69317+0.00000	test-corr:-0.02589+0.00211
[4]	train-logloss:0.69312+0.00000	train-corr:-0.05606+0.00093	test-logloss:0.69317+0.00000	test-corr:-0.02686+0.00246


In [3]:
from sklearn.datasets import load_boston
from sklearn.metrics import r2_score
boston=load_boston()
X =pd.DataFrame(boston.data,columns=boston.feature_names)
y = boston.target

In [4]:
df= X
df['Price'] = y
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,Price
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [5]:
dtrain = xgb.DMatrix(X, y, feature_names=X.columns.values)
def xgb_r2(preds, dtrain):
    labels = dtrain.get_label()
    return 'r2', r2_score(preds, labels)

In [11]:
def hyp_xgb(max_depth, subsample, colsample_bytree,min_child_weight, gamma ):
    params = {
#     'n_estimators': 300,
    'eta': 0.05,
    'objective': 'reg:squarederror',
    'eval_metric':'mae',
#     'silent': 1
     }
    params['max_depth'] = int(round(max_depth))
    params['subsample'] = max(min(subsample, 1), 0)
    params['colsample_bytree'] = max(min(colsample_bytree, 1), 0)
    params['min_child_weight'] = int(min_child_weight)
    params['gamma'] = max(gamma, 0)
    scores = xgb.cv(params,
                    dtrain,
                    num_boost_round=1000,
                    verbose_eval=False,
                    early_stopping_rounds=5,
                    feval=xgb_r2,
                    maximize=True,
                    nfold=5)
    return  scores['test-r2-mean'].iloc[-1]

In [12]:
pds ={
  'min_child_weight':(14, 20),
  'gamma':(0, 5),
  'subsample':(0.5, 1),
  'colsample_bytree':(0.1, 1),
  'max_depth': (6, 10)
}

In [13]:

# Surrogate model
optimizer = BayesianOptimization(hyp_xgb, pds, random_state=103)
                                  
# Optimize
optimizer.maximize(init_points=5, n_iter=15)

|   iter    |  target   | colsam... |   gamma   | max_depth | min_ch... | subsample |
-------------------------------------------------------------------------------------
|  1        |  0.9735   |  0.4889   |  0.8711   |  6.684    |  18.97    |  0.7936   |
|  2        |  0.9812   |  0.5134   |  4.113    |  9.286    |  15.84    |  0.6004   |
|  3        |  0.9746   |  0.4629   |  4.739    |  8.71     |  17.65    |  0.8362   |
|  4        |  0.8439   |  0.1072   |  1.683    |  7.429    |  16.92    |  0.9391   |
|  5        |  0.9901   |  0.7794   |  3.152    |  7.506    |  17.54    |  0.8687   |
|  6        |  0.9952   |  0.8073   |  0.3979   |  9.758    |  16.49    |  0.9983   |
|  7        |  0.978    |  0.5258   |  4.053    |  9.223    |  15.89    |  0.5299   |
|  8        |  0.9958   |  1.0      |  4.152    |  7.229    |  18.17    |  0.8302   |
|  9        |  0.9944   |  1.0      |  3.341    |  8.145    |  18.91    |  0.7549   |
|  10       |  0.9958   |  1.0      |  3.193    |  6.5

In [14]:
optimizer.max


{'target': 0.9984082000000001,
 'params': {'colsample_bytree': 1.0,
  'gamma': 4.435090227600583,
  'max_depth': 6.942298484826218,
  'min_child_weight': 16.757439116701462,
  'subsample': 1.0}}